<a href="https://colab.research.google.com/github/Satwikram/NLP-Implementations/blob/main/Sarcasm%20Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Author: Satwik Ram K

### Connecting to Kaggle

In [1]:
from google.colab import files

files.upload()


! mkdir ~/.kaggle


! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [2]:
!kaggle datasets download -d rmisra/news-headlines-dataset-for-sarcasm-detection

  0% 0.00/3.30M [00:00<?, ?B/s]
100% 3.30M/3.30M [00:00<00:00, 53.8MB/s]


In [3]:
!unzip /content/news-headlines-dataset-for-sarcasm-detection.zip

Archive:  /content/news-headlines-dataset-for-sarcasm-detection.zip
  inflating: Sarcasm_Headlines_Dataset.json  
  inflating: Sarcasm_Headlines_Dataset_v2.json  


### Importing Dependencies

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [5]:
dataset = pd.read_json("/content/Sarcasm_Headlines_Dataset_v2.json", lines = True)

In [6]:
dataset.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [7]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28619 entries, 0 to 28618
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   is_sarcastic  28619 non-null  int64 
 1   headline      28619 non-null  object
 2   article_link  28619 non-null  object
dtypes: int64(1), object(2)
memory usage: 670.9+ KB


In [8]:
dataset["headline"][0]

'thirtysomething scientists unveil doomsday clock of hair loss'

### Taking X and Y

In [9]:
X = dataset["headline"]

In [10]:
y = dataset["is_sarcastic"]

### Splitting Data into Train and Test

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [13]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

### Tokenization

In [14]:
vocab_size = 10000
embedding_dim = 16
max_length = 150

In [15]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token = "<oov>")

In [16]:
tokenizer.fit_on_texts(X_train)

In [17]:
print(tokenizer.word_index)

{'<oov>': 1, 'to': 2, 'of': 3, 'the': 4, 'in': 5, 'for': 6, 'a': 7, 'on': 8, 'and': 9, 'with': 10, 'is': 11, 'new': 12, 'trump': 13, 'man': 14, 'at': 15, 'from': 16, 'about': 17, 'by': 18, 'after': 19, 'you': 20, 'this': 21, 'up': 22, 'out': 23, 'be': 24, 'that': 25, 'as': 26, 'how': 27, 'it': 28, 'not': 29, 'he': 30, 'are': 31, 'just': 32, 'his': 33, 'your': 34, 'what': 35, 'all': 36, 'who': 37, 'has': 38, 'will': 39, 'into': 40, 'report': 41, 'more': 42, 'one': 43, 'have': 44, 'over': 45, 'year': 46, 'u': 47, 'area': 48, 'says': 49, 'why': 50, 'can': 51, 'day': 52, 's': 53, 'woman': 54, 'like': 55, 'first': 56, 'time': 57, 'donald': 58, 'no': 59, 'get': 60, 'her': 61, 'now': 62, 'old': 63, 'off': 64, "trump's": 65, "'": 66, 'people': 67, 'life': 68, 'obama': 69, 'house': 70, 'an': 71, 'white': 72, 'still': 73, 'back': 74, 'make': 75, 'than': 76, 'if': 77, 'was': 78, 'women': 79, 'down': 80, 'when': 81, 'i': 82, 'my': 83, 'clinton': 84, 'could': 85, '5': 86, 'way': 87, 'they': 88, 'th

In [18]:
train_sequence = tokenizer.texts_to_sequences(X_train)

In [19]:
train_padded = pad_sequences(train_sequence, maxlen = max_length, truncating = "post")

In [20]:
test_sequence = tokenizer.texts_to_sequences(X_test)

In [21]:
test_padded = pad_sequences(test_sequence, maxlen = max_length)

In [22]:
type(y_train)

numpy.ndarray

### Define Basic Neural Networks

In [23]:
model = tf.keras.models.Sequential([
                                    
        tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),

        tf.keras.layers.GlobalAveragePooling1D(),

        tf.keras.layers.Dense(512, activation = "relu"),

        tf.keras.layers.Dense(1, activation = "sigmoid")


])

In [24]:
model.compile(loss = "binary_crossentropy", metrics = ["accuracy"], optimizer = "adam")

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 150, 16)           160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 512)               8704      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 169,217
Trainable params: 169,217
Non-trainable params: 0
_________________________________________________________________


In [26]:
model.fit(train_padded, y_train, validation_data = (test_padded, y_test), epochs = 10)

Epoch 1/10
716/716 [==============================] - 8s 6ms/step - loss: 0.5700 - accuracy: 0.6797 - val_loss: 0.3889 - val_accuracy: 0.8241
Epoch 2/10
716/716 [==============================] - 4s 6ms/step - loss: 0.3235 - accuracy: 0.8649 - val_loss: 0.3495 - val_accuracy: 0.8417
Epoch 3/10
716/716 [==============================] - 4s 6ms/step - loss: 0.2580 - accuracy: 0.8960 - val_loss: 0.3347 - val_accuracy: 0.8519
Epoch 4/10
716/716 [==============================] - 4s 6ms/step - loss: 0.2165 - accuracy: 0.9138 - val_loss: 0.3411 - val_accuracy: 0.8538
Epoch 5/10
716/716 [==============================] - 4s 6ms/step - loss: 0.1849 - accuracy: 0.9282 - val_loss: 0.4154 - val_accuracy: 0.8346
Epoch 6/10
716/716 [==============================] - 5s 6ms/step - loss: 0.1674 - accuracy: 0.9357 - val_loss: 0.3924 - val_accuracy: 0.8482
Epoch 7/10
716/716 [==============================] - 5s 7ms/step - loss: 0.1459 - accuracy: 0.9452 - val_loss: 0.4195 - val_accuracy: 0.8487
Epoch 

### Classifying the new text

In [27]:
sentences = ["Oh is it?", "Silence is golden. Duct tape is silver."]

In [28]:
predict_sequences = tokenizer.texts_to_sequences(sentences)

In [29]:
padded_seq = pad_sequences(sequences = predict_sequences, maxlen = max_length, truncating = "post")

In [30]:
model.predict(padded_seq)

array([[0.0086363],
       [0.664812 ]], dtype=float32)

### Implementing LSTM

In [31]:
model1 = tf.keras.models.Sequential([
                                     
         tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),

         tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences = True)),

         tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),

         tf.keras.layers.Dense(64, activation = "relu"),

         tf.keras.layers.Dense(1, activation = "sigmoid")
])

In [32]:
model1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 16)           160000    
_________________________________________________________________
bidirectional (Bidirectional (None, 150, 128)          41472     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               98816     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 308,609
Trainable params: 308,609
Non-trainable params: 0
_________________________________________________________________


In [33]:
model1.compile(loss = "binary_crossentropy", metrics = ["accuracy"], optimizer = "adam")

In [34]:
model1.fit(train_padded, y_train, validation_data = (test_padded, y_test), epochs = 10)

Epoch 1/10
716/716 [==============================] - 119s 153ms/step - loss: 0.5487 - accuracy: 0.6962 - val_loss: 0.3768 - val_accuracy: 0.8367
Epoch 2/10
716/716 [==============================] - 108s 151ms/step - loss: 0.3001 - accuracy: 0.8763 - val_loss: 0.3525 - val_accuracy: 0.8499
Epoch 3/10
716/716 [==============================] - 108s 150ms/step - loss: 0.2125 - accuracy: 0.9178 - val_loss: 0.3720 - val_accuracy: 0.8456
Epoch 4/10
716/716 [==============================] - 108s 151ms/step - loss: 0.1662 - accuracy: 0.9387 - val_loss: 0.4918 - val_accuracy: 0.8377
Epoch 5/10
716/716 [==============================] - 108s 151ms/step - loss: 0.1313 - accuracy: 0.9548 - val_loss: 0.4719 - val_accuracy: 0.8389
Epoch 6/10
716/716 [==============================] - 108s 151ms/step - loss: 0.1057 - accuracy: 0.9641 - val_loss: 0.5486 - val_accuracy: 0.8438
Epoch 7/10
716/716 [==============================] - 108s 151ms/step - loss: 0.0783 - accuracy: 0.9757 - val_loss: 0.6350 -

In [36]:
model1.predict(padded_seq)

array([[0.00936779],
       [0.2414308 ]], dtype=float32)